In [1]:
import scanpy as sc

import warnings
warnings.filterwarnings("ignore")

In [2]:
import scTenifoldXct as st

In [3]:
# load scRNA-seq data
adata = sc.read_h5ad('data/adata_merge_example.h5ad')
adata

AnnData object with n_obs × n_vars = 199 × 2608
    obs: 'n_genes', 'percent_mito', 'n_counts', 'louvain', 'patient_id', 'patient_tumorsection', 'NormalvsTumor', 'patientno', 'PNC', 'PIC', 'ViralvsNonViral', 'ident', 'batch'
    var: 'gene_ids', 'n_cells'
    obsm: 'X_pca', 'X_umap'
    layers: 'log1p', 'raw'

In [4]:
# Normal vs Tumor samples
ada_N = adata[adata.obs['NormalvsTumor'] == 'N', :].copy()
ada_T = adata[adata.obs['NormalvsTumor'] == 'T', :].copy()
ada_N.shape, ada_T.shape

((101, 2608), (98, 2608))

In [6]:
# build a scTenifoldXct object
xct_N = st.scTenifoldXct(data = ada_N, 
                        source_celltype = 'B cells', 
                        target_celltype = 'Fibroblasts',
                        obs_label = "ident",
                        rebuild_GRN = False, # timer
                        GRN_file_dir = 'Net_example/Net_B2Fib_N/',  
                        verbose = True,
                        n_cpus = -1)

(cell, feature): (65, 2608)
(cell, feature): (36, 2608)
selected 64 LR pairs
load GRN B cells
load GRN Fibroblasts
build correspondence and initiate a trainer
scTenifoldXct init completed



In [7]:
xct_T = st.scTenifoldXct(data = ada_T, 
                        source_celltype = 'B cells', 
                        target_celltype = 'Fibroblasts',
                        obs_label = "ident",
                        rebuild_GRN = False, 
                        GRN_file_dir = 'Net_example/Net_B2Fib_T/',  
                        verbose = True,
                        n_cpus = 8)

(cell, feature): (39, 2608)
(cell, feature): (59, 2608)
selected 92 LR pairs
load GRN B cells
load GRN Fibroblasts
build correspondence and initiate a trainer
scTenifoldXct init completed



In [8]:
# merge two scTenifoldXct objects
XCTs = st.merge_scTenifoldXct(xct_T, xct_N)

merge samples and build correspondence
merge_scTenifoldXct init completed



In [ ]:
# # show model architecture
# XCTs.trainer.arch()

In [ ]:
# load model if trained
# XCTs.trainer.load_model_states('model_mergeXct')

In [9]:
# get embeddings
emb = XCTs.get_embeds(train = True)
emb.shape

training...: 100%|██████████| 3000/3000 [27:13<00:00,  1.84it/s]


(10432, 3)

In [ ]:
# # save model
# XCTs.trainer.save_model_states('model_mergeXct_B2Fib')

In [ ]:
# # show loss
# XCTs.plot_losses()

In [10]:
# get significant L-R pairs
XCTs.nn_aligned_diff(emb) 
xcts_pairs_diff = XCTs.chi2_diff_test()
xcts_pairs_diff

computing pair-wise euclidean distances...
computing pair-wise euclidean distances...
merged pair-wise distances

Total enriched: 2 / 6801664


,ligand,receptor,dist,correspondence,dist,correspondence,diff2,FC,p_val,q_val,enriched_rank
CCNA2_ITGA5,CCNA2,ITGA5,0.011020,0.897987,0.021239,0.0,0.000104,23.019939,0.000002,0.001746,1
CCNA2_ITGB5,CCNA2,ITGB5,0.010038,1.471727,0.018740,0.0,0.000076,16.690244,0.000044,0.014283,2
